In [7]:
import sqlite3
import json
import math
from datetime import datetime
import random
import time
from statistics import variance

In [ ]:
!pip install metno-locationforecast

# RETRIEVING HOME COORDINATES

In [12]:
def find_coordinates(db_path):
    coordinates = []
    
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute("SELECT shared_attrs FROM state_attributes")
        
        for row in cursor.fetchall():
            state = row[0]
            if state:
                state_dict = json.loads(state)
                
                if "latitude" in state_dict and "longitude" in state_dict:
                    lat = state_dict["latitude"]
                    lon = state_dict["longitude"]
                    
                    if not (lat == 0 and lon == 0):
                        coordinates.append((lat, lon))
    
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    
    finally:
        if conn:
            conn.close()
    
    return coordinates

db_path = ".homeassistant/home-assistant_v2.db"
coordinates = find_coordinates(db_path)
print("Retrieved coordinates:", coordinates)


Retrieved coordinates: [(51.1275572, 4.2147459), (51.1275572, 4.2147459)]


# ADDING NOISE TO COORDINATES

Met.no api truncates coordinates to 4 decimals
In Belgium, a difference of 0.0001 in latitude is 11.1m, in 0.0001 in longitude is 7.1m

WARNING: no more than 10 requests per second

In [20]:
from metno_locationforecast import Place, Forecast

home_coords = coordinates[0]
#truncate to 4 decimals
home_coords = (float(f"{home_coords[0]:.4f}"),float(f"{home_coords[1]:.4f}"))

user_agent = "PrivacyTesting/1.0 (pieter-jan.vancouwenberghe@student.kuleuven.be)"

desired_time = datetime(2024, 11, 24, 12, 0, 0)
noise_variances = [0.1,0.5,1,2]
num_trials_per_variance = 20

results = {}

def get_temperature(lat, lon):
    place = Place(
        name="Home",
        latitude=lat,
        longitude=lon,
        altitude=0
    )
    forecast = Forecast(place, user_agent)
    forecast.update()
    time.sleep(0.5)  # avoid overloading API

    for interval in forecast.data.intervals:
        if interval.start_time == desired_time:
            return interval.variables["air_temperature"].value
    return None

for var in noise_variances:
    temperatures = []
    print("### VARIANCE: "+str(var)+" ###")

    for _ in range(num_trials_per_variance):
        # add noise to coordinates
        noisy_lat = home_coords[0] + random.gauss(0, var)
        noisy_lon = home_coords[1] + random.gauss(0, var)
        
        temp = get_temperature(noisy_lat, noisy_lon) 
        temperatures.append(temp)
        print("lat:"+str(noisy_lat)+" lon: "+str(noisy_lon)+" temp: "+str(temp))

    if len(temperatures) > 1:
        temp_variance = variance(temperatures)
    else:
        temp_variance = 0
        
    results[var] = temp_variance

print("\nTemperature variance results:")
for var, temp_var in results.items():
    print(f"Noise Variance: {var}, Temperature Variance: {temp_var}")

### VARIANCE: 0.1 ###
lat:51.14393048469217 lon: 4.401824222269102 temp: 17.6
lat:51.06142802722937 lon: 4.13355848292432 temp: 17.9
lat:51.08387547365361 lon: 4.358761394381707 temp: 17.6
lat:51.04603869035031 lon: 4.26675017381676 temp: 17.7
lat:51.04045087884305 lon: 4.166928786047102 temp: 17.8
lat:51.06528411362378 lon: 4.217651706523089 temp: 17.8
lat:51.012169790701634 lon: 4.171633586345948 temp: 17.8
lat:51.042620345783824 lon: 4.058064684942208 temp: 17.8
lat:51.120263937614396 lon: 4.149269026656576 temp: 17.9
lat:51.08125436761929 lon: 4.227654111838927 temp: 17.8
lat:51.04307871514265 lon: 4.174118813784737 temp: 17.8
lat:51.12469519013219 lon: 4.409379945450259 temp: 17.6
lat:50.81249588940706 lon: 4.285211681271907 temp: 17.3
lat:51.119573279346554 lon: 4.061627566877585 temp: 17.9
lat:51.09614226468557 lon: 4.2403235172058675 temp: 17.8
lat:51.219947583976804 lon: 4.210255838122562 temp: 17.6
lat:51.08225480674182 lon: 4.29501875111862 temp: 17.7
lat:51.15158447498833 l

# ENCRYPTION OF LOCATION DATA

In [1]:
!pip install pycryptodome

     ---------------------------------------- 1.8/1.8 MB 7.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import os

original_db_path = ".homeassistant/home-assistant_v2.db"
encrypted_db_path = ".homeassistant/home-assistant_v2_encrypted.db"

key = get_random_bytes(32)  # =AES-256
iv = get_random_bytes(16)   # initalization vector

In [4]:
def encrypt_file(input_file, output_file, key, iv):
    cipher = AES.new(key, AES.MODE_CFB, iv)
    with open(input_file, 'rb') as f:
        plaintext = f.read()
    ciphertext = cipher.encrypt(plaintext)
    with open(output_file, 'wb') as f:
        f.write(iv) 
        f.write(ciphertext)
    print(f"Encrypted {input_file} -> {output_file}")

encrypt_file(original_db_path, encrypted_db_path, key, iv)

Encrypted .homeassistant/home-assistant_v2.db -> .homeassistant/home-assistant_v2_encrypted.db


In [17]:
encrypted_coordinates = find_coordinates(".homeassistant/home-assistant_v2_encrypted.db")

print(encrypted_coordinates)

SQLite error: file is not a database
[]


In [5]:
def decrypt_file(input_file, output_file, key):
    with open(input_file, 'rb') as f:
        iv = f.read(16)  # Read the IV
        ciphertext = f.read()
    cipher = AES.new(key, AES.MODE_CFB, iv)
    plaintext = cipher.decrypt(ciphertext)
    with open(output_file, 'wb') as f:
        f.write(plaintext)
    print(f"Decrypted {input_file} -> {output_file}")

decrypted_db_path = ".homeassistant/home-assistant_v2_decrypted.db"
decrypt_file(encrypted_db_path, decrypted_db_path, key)


Decrypted .homeassistant/home-assistant_v2_encrypted.db -> .homeassistant/home-assistant_v2_decrypted.db


In [18]:
decrypted_coordinates = find_coordinates(".homeassistant/home-assistant_v2_decrypted.db")

print(decrypted_coordinates)

[(51.1275572, 4.2147459), (51.1275572, 4.2147459)]
